# Simple ETL Application

In this notebook, we connect to an api, extract data, create a spark dataframe and then finally write to a postgres database. 

In [1]:
import requests
import json
import pandas as pd
import psycopg2
from dags.src.sql_base import sql_base
from dags.src.fetch_data import create_dataframe
from dags.src.get_data import get_data

# Get Data

In [2]:
_,record = get_data()

In [3]:
record

{'gender': 'male',
 'name': {'title': 'Mr', 'first': 'Marvin', 'last': 'Howard'},
 'location': {'street': {'number': 1571, 'name': 'Galway Road'},
  'city': 'Monaghan',
  'state': 'Limerick',
  'country': 'Ireland',
  'postcode': 75833,
  'coordinates': {'latitude': '-3.3901', 'longitude': '-63.7871'},
  'timezone': {'offset': '-1:00',
   'description': 'Azores, Cape Verde Islands'}},
 'email': 'marvin.howard@example.com',
 'login': {'uuid': 'a6622a8b-2238-4b63-ae8a-62eb2e466bab',
  'username': 'lazymouse481',
  'password': 'todd',
  'salt': 'jjoHTAoi',
  'md5': '832e8f143320d62814bff3ace2ac6ab3',
  'sha1': '49e0b2e267058f9b119ce2f951daee713e9453fa',
  'sha256': '6d7193865f389f6d48ad365b695cd2a3722b75adae9314911c036851cd7f783c'},
 'dob': {'date': '1954-02-22T21:41:26.875Z', 'age': 70},
 'registered': {'date': '2020-09-26T09:47:15.683Z', 'age': 3},
 'phone': '021-924-4767',
 'cell': '081-510-6361',
 'id': {'name': 'PPS', 'value': '0333461T'},
 'picture': {'large': 'https://randomuser.me

In [4]:
db = sql_base()

In [5]:
db.write_record_to_table(record)

'INSERT INTO users.people(name,surname,title,gender,country,city,coordinates) VALUES(\'Marvin\', \'Howard\', \'Mr\', \'male\', \'Ireland\', \'Monaghan\', \'{"latitude": "-3.3901", "longitude": "-63.7871"}\')'

In [7]:
record

(200,
 {'gender': 'male',
  'name': {'title': 'Monsieur', 'first': 'Jean-Daniel', 'last': 'Robin'},
  'location': {'street': {'number': 6703, 'name': 'Quai Charles-De-Gaulle'},
   'city': 'Bellmund',
   'state': 'Genève',
   'country': 'Switzerland',
   'postcode': 3978,
   'coordinates': {'latitude': '-59.9145', 'longitude': '-154.9542'},
   'timezone': {'offset': '-5:00',
    'description': 'Eastern Time (US & Canada), Bogota, Lima'}},
  'email': 'jean-daniel.robin@example.com',
  'login': {'uuid': '7e4a514c-3a35-41a6-ab86-eb8d5f96e0bf',
   'username': 'lazygorilla909',
   'password': 'qaz123',
   'salt': 'pOouhOAN',
   'md5': 'd7785bc804aa909767b713e019a95129',
   'sha1': 'dcff01ee054a8ac03c620e637f97398bc87768c7',
   'sha256': 'e4b6dc780cb5745064f248911c57a694d4853aa3545fd435e56a14d753c07d8a'},
  'dob': {'date': '1984-05-21T02:32:38.364Z', 'age': 39},
  'registered': {'date': '2012-06-08T12:11:06.298Z', 'age': 11},
  'phone': '079 539 07 97',
  'cell': '076 917 02 80',
  'id': {'na

In [6]:
statement = "insert into users.people (name) values ('mukovhe')"

In [7]:
db.sql_runner(statement)